# 프로젝트 - Movielens 영화 추천 실습

## 1) 데이터 준비와 전처리

In [33]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [34]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [35]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [36]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2) 분석해 봅시다.

### · ratings에 있는 유니크한 영화 개수

In [37]:
ratings['movie_id'].nunique()

3628

### · ratings에 있는 유니크한 사용자 수

In [38]:
ratings['user_id'].nunique()

6039

### · 가장 인기 있는 영화 30개(인기순)

In [74]:
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [40]:
my_favorite = [2, 4, 6, 8, 10]
my_playlist = pd.DataFrame({'user_id': [6041]*5, 'movie_id': my_favorite, 'counts':5,'timestamp':0})
ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,timestamp
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569
0,6041,2,5,0
1,6041,4,5,0
2,6041,6,5,0
3,6041,8,5,0
4,6041,10,5,0


In [44]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

## 4) CSR matrix를 직접 만들어 봅시다.

In [48]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].max()+1
num_movie = ratings['movie_id'].max()+1
#num_user = ratings['user_id'].nunique()
#num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [49]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [50]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [51]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [54]:
me, jumanji = user_to_idx[6041], movie_to_idx[2]
me_vector, jumanji_vector = als_model.user_factors[me], als_model.item_factors[jumanji]

In [55]:
me_vector

array([ 0.6280567 , -0.38646492, -1.9249898 , -0.04649313, -1.2184331 ,
        0.5254717 ,  2.6878674 ,  0.659227  , -2.1366112 , -4.0664015 ,
       -1.4650328 , -1.581714  , -0.74193764, -0.6219538 ,  0.87858856,
       -1.6507298 , -0.08209006, -0.5705416 , -0.3630629 ,  2.2237635 ,
        3.815673  ,  0.31418234, -0.3338307 ,  1.8074039 ,  1.0850616 ,
       -1.9200345 ,  4.3813486 , -0.3204128 , -0.8519948 ,  3.6501098 ,
        0.33271825,  4.5410676 ,  1.1607233 ,  0.09208515,  1.6568532 ,
       -1.3396477 ,  1.776712  , -2.5868    ,  0.10112229, -0.4166587 ,
        0.3540366 , -1.2650297 ,  1.8914284 ,  3.1717744 ,  0.293455  ,
        2.672774  , -2.3721502 , -1.1038758 ,  1.2670362 ,  0.56853956,
        0.25345117,  1.0983026 ,  3.3000445 , -2.799539  , -4.363908  ,
        1.4966503 , -1.0555766 , -0.18828174, -0.17955805, -1.106352  ,
       -1.3545285 , -1.0924753 ,  1.1778096 ,  0.8752473 ,  0.45802212,
        0.77473813,  1.6354551 ,  1.6400841 , -1.617556  , -1.03

In [56]:
jumanji_vector

array([ 1.0481494e-02,  1.6185994e-03,  1.2045318e-02,  9.3268667e-04,
        1.0635717e-02,  1.2367192e-03, -1.2142613e-03,  4.9519073e-03,
        3.6485018e-03,  4.9356944e-03,  6.4152493e-03,  3.3075719e-03,
        2.3900094e-03,  3.3526516e-03,  4.9857586e-03,  8.3616404e-03,
        1.1085597e-02,  8.3494270e-03,  1.1502509e-02, -9.9077661e-05,
        1.0702958e-02,  3.5270317e-03,  7.3934444e-03,  6.4276122e-03,
        5.5754097e-04,  9.2654498e-03,  1.1212640e-02,  1.0361959e-02,
        6.3855276e-03,  8.1750723e-03,  1.0148946e-02,  9.7567160e-03,
        2.2809925e-03,  1.1861688e-02,  5.9397747e-03,  9.5319301e-03,
        9.5472215e-03,  5.2676722e-03,  2.1547233e-03,  9.1772396e-03,
        1.1692219e-02,  1.3606133e-02,  9.7890915e-03,  8.9877993e-03,
        6.2156445e-03,  8.4554330e-03,  6.1887233e-03,  1.4202192e-02,
        1.1723966e-02,  1.5222238e-03,  2.5673752e-04,  2.4907994e-03,
        9.6156457e-03,  5.9438744e-03, -7.1880512e-04,  3.9838194e-03,
      

In [57]:
np.dot(me_vector, jumanji_vector)

0.15731911

In [58]:
grumpier = movie_to_idx[3]
grumpier_vector = als_model.item_factors[grumpier]
np.dot(me_vector, grumpier_vector)

-0.034784187

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [75]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

def get_similar_movies(movie_id: int):
    similar_movie = als_model.similar_items(movie_id)
    return similar_movie

idx_to_title = {movies.loc[i,'movie_id']:movies.loc[i,'title'] for i in range(len(movies))}
[[idx_to_title.get(i[0]),i[1]] for i in get_similar_movies(2)]

[['Jumanji (1995)', 0.99999994],
 ['Hook (1991)', 0.84063315],
 ['Indian in the Cupboard, The (1995)', 0.8221478],
 ['Dragonheart (1996)', 0.75426394],
 ['Santa Clause, The (1994)', 0.6850581],
 ['Space Jam (1996)', 0.6745924],
 ['Flubber (1997)', 0.655165],
 ['Small Soldiers (1998)', 0.6530893],
 ['NeverEnding Story II: The Next Chapter, The (1990)', 0.64165366],
 ['Borrowers, The (1997)', 0.61901635]]

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [76]:
user = user_to_idx[6041]
# recommend에서는 user*item CSR Matrix를 받습니다.
movies_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movies_recommended

[[idx_to_title.get(i[0]), i[0]] for i in movies_recommended]

[['Some Like It Hot (1959)', 910],
 ['Holiday Inn (1942)', 3061],
 ['Philadelphia Story, The (1940)', 898],
 ['Sound of Music, The (1965)', 1035],
 ['Blazing Saddles (1974)', 3671],
 ['White Christmas (1954)', 3675],
 ['Snow White and the Seven Dwarfs (1937)', 594],
 ['It Happened One Night (1934)', 905],
 ['North by Northwest (1959)', 908],
 ['Star Wars: Episode V - The Empire Strikes Back (1980)', 1196],
 ['Manchurian Candidate, The (1962)', 1267],
 ['Rebecca (1940)', 928],
 ['Monty Python and the Holy Grail (1974)', 1136],
 ['Young Frankenstein (1974)', 1278],
 ['Double Indemnity (1944)', 3435],
 ['Strangers on a Train (1951)', 2186],
 ['Duck Soup (1933)', 1256],
 ['On the Town (1949)', 3606],
 ['Jungle Book, The (1967)', 2078],
 ['Sting, The (1973)', 1234]]

In [77]:
explain = als_model.explain(user, csr_data, itemid=910)

[(idx_to_title.get(i[0]), i[1]) for i in explain[1]]

[('Producers, The (1968)', 0.10782674441084628),
 ('To Catch a Thief (1955)', 0.06620059102493549),
 ('Roman Holiday (1953)', 0.06275205015884137),
 ('Cool Hand Luke (1967)', 0.06237457163949778),
 ('His Girl Friday (1940)', 0.05118136748463717),
 ('Mister Roberts (1955)', 0.05026925484128814),
 ('Bringing Up Baby (1938)', 0.049737074884179784),
 ('Vertigo (1958)', 0.048212058518972985),
 ('Rear Window (1954)', 0.04384037467038282),
 ('My Fair Lady (1964)', 0.04325133616195797)]